In [ ]:
import pandas as pd
import numpy as np
from pycaret.classification import * # Machine learning tools
from sklearn.metrics import ConfusionMatrixDisplay # Model evaluation
import matplotlib.pyplot as plt # Visualization
import seaborn as sns
from sklearn.metrics import classification_report # Model evaluation report
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
import matplotlib.pyplot as plt
from collections import Counter
import shap
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
training_data = pd.read_excel('/home/dsg/vortex/PRODUCTION/DATA/processed/training_data.xlsx', engine='openpyxl')
training_data.info()

In [ ]:
# Experiment setup 

experiment = setup(data=training_data, target= 'Site',train_size=0.8, session_id=123)

In [ ]:
remove_metric('MCC')
remove_metric('Kappa')
remove_metric('AUC')
#'AUC', , 'MCC'

In [ ]:
rf = create_model('rf', class_weight="balanced", criterion='entropy')

In [ ]:
tuned_model = tune_model(rf, n_iter=10, tuner_verbose=True, optimize='F1', custom_grid = {'criterion': ['entropy']})

### Calculating feature importance using RFECV

In [ ]:
training_data = get_config('X_train')
y_training= get_config('y_train')
training_data['Site'] = y_training
y_training

In [ ]:
train = get_config('X_train')
train.columns

In [ ]:
rvcdf = training_data  

# Separar características (X) y variable objetivo (y)
X = rvcdf.drop(['Site', 'rand'], axis=1)
y = rvcdf['Site']
rvcdf['Site'].value_counts()

In [ ]:
X.columns

In [ ]:
#RFECV loop
def run_rfecv(X, y, tuned_model, random_state):
    np.random.seed(random_state)  # Set seed for any numpy operations
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)
    rfecv = RFECV(estimator=clone(tuned_model), step=2, cv=cv, scoring='accuracy')
    rfecv.fit(X, y)
    
    # Handle different scikit-learn versions
    if hasattr(rfecv, 'cv_results_'):
        # For newer scikit-learn versions
        grid_scores = rfecv.cv_results_['mean_test_score']
    elif hasattr(rfecv, 'grid_scores_'):
        # For older scikit-learn versions
        grid_scores = rfecv.grid_scores_
    else:
        raise AttributeError("RFECV object has neither 'cv_results_' nor 'grid_scores_' attribute")
    
    # Get the mask of selected features
    feature_mask = rfecv.support_
    
    return rfecv.n_features_, grid_scores, feature_mask

# Number of times to run the process
n_runs = 10

# Lists to store results
n_features_list = []
grid_scores_list = []
feature_masks = []

# Run the process multiple times
for i in range(n_runs):
    n_features, grid_scores, feature_mask = run_rfecv(X, y, tuned_model, random_state=i)
    n_features_list.append(n_features)
    grid_scores_list.append(grid_scores)
    feature_masks.append(feature_mask)

# Calculate average and variance of optimal number of features
avg_n_features = np.mean(n_features_list)
var_n_features = np.var(n_features_list)

print(f"Average optimal number of features: {avg_n_features:.2f}")
print(f"Variance in optimal number of features: {var_n_features:.2f}")

# Calculate average and variance of grid scores
avg_grid_scores = np.mean(grid_scores_list, axis=0)
var_grid_scores = np.var(grid_scores_list, axis=0)


# Print the range of optimal features
min_features = min(n_features_list)
max_features = max(n_features_list)
print(f"Range of optimal number of features: {min_features} to {max_features}")
feature_selection_counts = Counter()
for mask in feature_masks:
    feature_selection_counts.update(np.where(mask)[0])

total_runs = len(feature_masks)
consistent_features = [feature for feature, count in feature_selection_counts.items() 
                       if count == total_runs]
mostly_consistent_features = [feature for feature, count in feature_selection_counts.items() 
                              if count >= total_runs * 0.8]

# Map indices to feature names
consistent_feature_names = [feature_names[i] for i in consistent_features]
mostly_consistent_feature_names = [feature_names[i] for i in mostly_consistent_features]

print("\nFeatures selected in all runs:")
for idx, name in zip(consistent_features, consistent_feature_names):
    print(f"Index {idx}: {name}")

print("\nFeatures selected in at least 80% of runs:")
for idx, name in zip(mostly_consistent_features, mostly_consistent_feature_names):
    print(f"Index {idx}: {name}")



In [ ]:
# Importar el backend de Matplotlib
import matplotlib


# Modificación en la parte de visualización
fig, ax = plt.subplots(figsize=(10, 5))

# Transponer grid_scores_list para que cada columna represente un número de características
grid_scores_transposed = list(map(list, zip(*grid_scores_list)))

# Crear el boxplot
#bp = ax.boxplot(grid_scores_transposed, positions=range(1, len(avg_grid_scores) + 1), 
#                widths=0.6, patch_artist=True)

# Personalizar el color de los boxplots
#for box in bp['boxes']:
#    box.set(facecolor='lightblue', edgecolor='blue', alpha=0.7)

# Calcular los límites superior e inferior para la región sombreada
x = range(1, len(avg_grid_scores) + 1)
lower_bound = avg_grid_scores - np.sqrt(var_grid_scores)
upper_bound = avg_grid_scores + np.sqrt(var_grid_scores)

# Dibujar la región sombreada
ax.fill_between(x, lower_bound, upper_bound, color='gray', alpha=0.3)

# Dibujar la línea de promedios
ax.plot(x, avg_grid_scores, 'o-', color='gray', linewidth=1, markersize=4)

# Configurar el gráfico
ax.set_xlabel('Number of Features',fontweight='bold', fontsize=14)
ax.set_ylabel('Cross-validation Accuracy',fontweight='bold', fontsize=14)
#ax.set_title('RFECV Results: Distribution and Average Across Runs', fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=12)
ax.set_xticks(range(1, len(avg_grid_scores) + 1))
ax.set_xticklabels(range(1, len(avg_grid_scores) + 1))

# Añadir una leyenda
ax.plot([], [], 'o-', color='gray', linewidth=1, markersize=4, label='Average Score')
ax.fill([], [], color='gray', alpha=0.3, label='Standard Deviation')
#ax.plot([], [], 'b-', label='Median Score')
#ax.fill([], [], color='lightblue', alpha=0.7, label='Score Distribution')
ax.legend(fontsize=12)

ax.grid(True, linestyle='--', alpha=0.3)

fig.tight_layout()

# Guardar la figura
plt.savefig("/home/dsg/vortex/PRODUCTION/outputs/plots/RFECV_shaded_.png", dpi=300, bbox_inches='tight')

# Cerrar la figura para liberar memoria
plt.close(fig)


In [ ]:

feature_names = X.columns
def plot_feature_importance_dotplot(model, feature_names=None, n_features=10):
    # Obtener la importancia de las características
    importances = model.feature_importances_
    
    # Si no se proporcionan nombres de características, usar índices
    if feature_names is None:
        feature_names = [f'Feature {i}' for i in range(len(importances))]
    
    # Crear un DataFrame con las características y sus importancias
    feature_importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    })
    
    # Ordenar el DataFrame por importancia descendente
    feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)
    
    # Limitar el número de características a visualizar
    feature_importance_df = feature_importance_df.head(n_features)
    
    # Crear el gráfico de puntos
    fig, ax = plt.subplots(figsize=(10, max(6, n_features * 0.3)))
    
    # Dibujar líneas horizontales
    ax.hlines(y=range(n_features), xmin=0, xmax=feature_importance_df['importance'], color='skyblue')
    
    # Dibujar puntos
    ax.plot(feature_importance_df['importance'], range(n_features), "o")
    
    # Configurar el eje y
    ax.set_yticks(range(n_features),)
    ax.set_yticklabels(feature_importance_df['feature'])
    ax.invert_yaxis()  # Las características más importantes arriba
    
   
    ax.set_xlabel('Feature Importance')
    ax.set_ylabel('Features')
    # Añadir título y ajustar diseño
    #ax.set_title('Feature Importance Plot')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.grid(axis='x', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.xlabel('Feature Importance', fontsize=14)
    plt.ylabel('Features', fontsize=14)
    plt.yticks(fontsize=14)
    plt.xticks(fontsize=14)
    plt.savefig('/home/dsg/vortex/PRODUCTION/outputs/plots/14importantFeatures.png', dpi=300, bbox_inches='tight')
    plt.show()



# Ejemplo de uso:
# Asumiendo que ya tiene su modelo entrenado llamado 'rf_model' y una lista de nombres de características 'feature_names'
plot_feature_importance_dotplot(tuned_model, feature_names, n_features=14)

### Shapley Values

In [ ]:
import shap
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

def custom_interpret_model(model, X_train, plot='summary', feature_name=None, observation=None, 
                           use_train_data=True, X_new_sample=None, class_names=None, max_display=None,
                           save_path=None):
    
    # Obtener el objeto de explicación SHAP
    explainer = shap.TreeExplainer(model)
    
    # Usar X_train si use_train_data es True, de lo contrario usar X_new_sample si se proporciona
    if use_train_data:
        data_for_shap = X_train
    elif X_new_sample is not None:
        data_for_shap = X_new_sample
    else:
        raise ValueError("Debe proporcionar X_new_sample si use_train_data es False")

    shap_values = explainer.shap_values(data_for_shap)

    # Crear una nueva figura
    plt.figure(figsize=(10, 8))

    # Crear una paleta secuencial de grises personalizada
    colors = [(0.6, 0.6, 0.6), (0.1, 0.1, 0.1)]  # Del gris claro al gris oscuro
    n_bins = 3  # Número de tonos de gris
    gray_cmap = LinearSegmentedColormap.from_list("custom_grays", colors, N=n_bins)

    # Generar el summary plot con la paleta de grises personalizada
    shap.summary_plot(shap_values, 
                      data_for_shap,
                      plot_type="bar",
                      class_names=class_names,
                      color=gray_cmap,
                      max_display=max_display,
                      show=False)  # No mostrar el gráfico inmediatamente

    # Ajustar el diseño
    plt.tight_layout()

    # Guardar o mostrar el gráfico
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()  # Cerrar la figura para liberar memoria
        print(f"Gráfico guardado en: {save_path}")
    else:
        plt.show()

# Ejemplo de uso de la función
custom_interpret_model(tuned_model, 
                        X_train,  # Sus datos de entrenamiento
                        class_names=['Gavá', 'Terena', 'Aliste'], 
                        max_display=14,
                        save_path='/home/dsg/vortex/PRODUCTION/outputs/plots/SHAPSummary.png')